In [8]:
from Laplace_JAXDense import solve_and_loss


In [1]:
# -*- coding: utf-8 -*-
'''
Created on Oct 2023

@authors:   Jamie Taylor (CUNEF)
            Manuela Bastidas (UPV/EHU)
            Tomas Teijeiro (BCAM)
            https://www.mathmode.science/
            
Updated on Jan 2025 by:
    Ángel Javier Omella
'''

# This code presents a simple implementation of Physics-Informed
# Neural Networks (PINNs) as a collocation method. -- <50 lines of PINNS--

# In this 1D example, we utilize Keras for constructing neural networks
# and JAX in the backend.


import jax
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt

import os

os.environ["KERAS_BACKEND"] = "jax"

import keras

In [23]:

# Set the random seed
np.random.seed(1234)
keras.utils.set_random_seed(1234)

dtype='float64' # double precision set to default in the SCR functions
jax.config.update("jax_enable_x64", True)
keras.backend.set_floatx(dtype)


# =============================================================================
#
#          Source code - PINNs H01 1D
#
# =============================================================================


## Define an approximate solution (u_nn): A neural network model
def make_model(neurons, n_layers, n_output, activation='tanh'):

    """
    Creates a neural network model to approximate the solution of
        int (grad u ).(grad v) - int f.v = 0

    Args:
        neurons (int): The number of neurons in each hidden layer.
        activation (str, optional): Activation function for hidden layers.

    Returns:
        keras.Model: A neural network model for the approximate solution.
    """

    xvals = keras.layers.Input(shape=(1,), name='x_input',dtype=dtype)

	# The input
    l1 = keras.layers.Dense(neurons, activation=activation, dtype=dtype)(xvals)

    ## ---------------
    #  The dense layers
    ## ---------------

    # First layer
    for l in range(n_layers-2):
        # Hidden layers
        l1 = keras.layers.Dense(neurons, activation=activation, dtype=dtype)(l1)
    # Last layer
    output = keras.layers.Dense(n_output, activation=activation, dtype=dtype)(l1)

    model = keras.Model(inputs = xvals, outputs = output, name='model')

    # Print the information of the model u
    model.summary()

    return model

In [25]:
model = make_model(neurons=10, n_output=10, n_layers=1)

Model: "model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ x_input (InputLayer)            │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 10)             │            20 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 10)             │           110 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 130 (1.02 KB)

 Trainable params: 130 (1.02 KB)

 Non-trainable params: 0 (0.00 B)

In [27]:
output = model(jnp.array([0]))

In [39]:
[sum(output[0][0:i]) for i in range(1,len(output[0]))]

[Array(0.10154107, dtype=float64),
 Array(0.21510571, dtype=float64),
 Array(0.31261436, dtype=float64),
 Array(0.41227527, dtype=float64),
 Array(0.51063267, dtype=float64),
 Array(0.59444266, dtype=float64),
 Array(0.68967052, dtype=float64),
 Array(0.79019495, dtype=float64),
 Array(0.89303934, dtype=float64)]